<a href="https://colab.research.google.com/github/vindruid/yolov3-in-colab/blob/master/yolov3_train_hand_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source :https://github.com/ultralytics/yolov3/wiki/Example:-Train-Single-Class

CHANGE RUNTIME USING GPU

In [0]:
!pip install wget

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image, ImageFile, ImageDraw, ImageFont
import requests
from io import BytesIO
import numpy as np
import os

import wget
import tarfile
import io
import hashlib
import scipy.io
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from tqdm import tqdm
from shutil import copyfile


# Prepare Dataset Yolo Format

In [0]:
def download_extract_data(data_filenames):
    # Load train, test and validation data
    for data_filename in data_filenames:

        print('Downloading %s.tar.gz...' % data_filename)
        data_url = 'http://www.robots.ox.ac.uk/~vgg/data/hands/downloads/%s.tar.gz' % data_filename
        data_filename_ext = wget.download(data_url)

        print('Extracting %s...' % data_filename_ext)
        data_tar = tarfile.open('%s' % data_filename_ext)
        data_tar.extractall(path='dataset/')
        data_tar.close()

# download_extract_data(['test_dataset', 'validation_dataset', 'training_dataset'])
download_extract_data(['test_dataset', 'training_dataset'])


In [0]:
DATA_PATH = './dataset'

# Test dataset
TEST_IMG_DIR = os.path.join(DATA_PATH, 'test_dataset', 'test_data', 'images')
TEST_ANN_DIR = os.path.join(DATA_PATH, 'test_dataset', 'test_data', 'annotations')

# Training dataset
TRAIN_IMG_DIR = os.path.join(DATA_PATH, 'training_dataset', 'training_data', 'images')
TRAIN_ANN_DIR = os.path.join(DATA_PATH, 'training_dataset', 'training_data', 'annotations')


In [0]:
class HandDatasetOxford(Dataset):
  def __init__(self, image_dir, ann_dir):
    self.transforms = transforms
    self.image_dir = image_dir
    self.image_list = sorted([filename for filename in os.listdir(self.image_dir) if ( (filename[-4:] == '.jpg') & (filename.split('.')[0].split('_')[0] == 'VOC2010') )])
    self.ann_dir = ann_dir 
    self.ann_list = sorted([filename for filename in os.listdir(self.ann_dir) if ( (filename[-4:] == '.mat') & (filename.split('.')[0].split('_')[0] == 'VOC2010') )])

  def __len__(self):
    return len(self.image_list)

  def __getitem__(self, idx):
    filename = self.image_list[idx]
    img_path = os.path.join(self.image_dir, filename)
    img = Image.open(img_path)
    boxes_path = os.path.join(self.ann_dir, self.ann_list[idx])
    boxes = self.boxes_from_mat(boxes_path)
    label = self.yolo_label_from_boxes(img, boxes) #exercise
    label_filename = filename.replace('.jpg', '.txt')

    sample = {}
    sample['image'] = img
    sample['label'] = label

    sample['img_filename'] = filename
    sample['label_filename'] = label_filename

    sample['image_path'] = img_path

    return sample

  #exercise
  def yolo_label_from_boxes(self, img, boxes): 
    W, H = img.size

    label = ""
    for box in boxes: 
      box_w, box_h = (box[2] - box[0]) / W, (box[3] - box[1]) / H 
      x_center, y_center = ((box[2] + box[0]) / 2) / W, ((box[3] + box[1]) / 2) / H

      # ensure value between 0 and 1 
      box_w = max(min(box_w, 1),0)
      box_h = max(min(box_h, 1),0)
      x_center = max(min(x_center, 1),0)
      y_center = max(min(y_center, 1),0)
      

      row = f"0 {x_center} {y_center} {box_w} {box_h}"
      label += row 
      label += "\n"
    label = label.strip()
      
    return label


  def boxes_from_mat(self, mat_filepath):
    mat = scipy.io.loadmat(mat_filepath)
    coords = []
    i = 0
    for e in mat['boxes'][0]:
        coords.append(list())
        c = 0
        for d in e[0][0]:
            if c > 3:
                break
            coords[i].append((d[0][0], d[0][1]))
            c += 1
        i += 1

    boxes = []
    for coord in coords:
      x_max, x_min, y_max, y_min = 0, float('inf'), 0, float('inf')
      for y,x in coord:
        x_max, x_min = np.max([x, x_max]), np.min([x, x_min])
        y_max, y_min = np.max([y, y_max]), np.min([y, y_min])

      box = [x_min, y_min, x_max, y_max]
      boxes.append(box)

    return boxes


In [0]:
train_dataset = HandDatasetOxford(TRAIN_IMG_DIR, TRAIN_ANN_DIR)
test_dataset = HandDatasetOxford(TEST_IMG_DIR, TEST_ANN_DIR)

In [0]:
path_images_train = os.path.join('hand', 'images', 'train')
path_images_test = os.path.join('hand', 'images', 'test')
path_labels_train = os.path.join('hand', 'labels', 'train')
path_labels_test = os.path.join('hand', 'labels', 'test')

In [0]:
!mkdir -p {path_images_train}
!mkdir -p {path_images_test}
!mkdir -p {path_labels_train}
!mkdir -p {path_labels_test}

In [0]:
for idx in tqdm(range(150)):
  sample = train_dataset[idx]

  label = sample['label']
  label_filename = sample['label_filename']
  img_filename = sample['img_filename']

  path_image_new = os.path.join(path_images_train, img_filename)
  path_label_new = os.path.join(path_labels_train, label_filename)

  # save label
  with open(path_label_new, "w") as f :
    f.write(label)

  copyfile(sample['image_path'], path_image_new)
  

for idx in tqdm(range(50)):
  sample = test_dataset[idx]

  label = sample['label']
  label_filename = sample['label_filename']
  img_filename = sample['img_filename']

  path_image_new = os.path.join(path_images_test, img_filename)
  path_label_new = os.path.join(path_labels_test, label_filename)

  # save label
  with open(path_label_new, "w") as f :
    f.write(label)

  copyfile(sample['image_path'], path_image_new)

In [0]:
###
str_hand_1cls_train = ""
for filename in os.listdir(path_images_train):
  row = str(os.path.join("..", path_images_train, filename))
  str_hand_1cls_train += row 
  str_hand_1cls_train += "\n"

str_hand_1cls_train = str_hand_1cls_train.strip()

with open('hand_1cls_train.txt', "w") as f :
  f.write(str_hand_1cls_train)

### 
str_hand_1cls_test = ""
for filename in os.listdir(path_images_test):
  row = str(os.path.join("..", path_images_test, filename))
  str_hand_1cls_test += row 
  str_hand_1cls_test += "\n"

str_hand_1cls_test = str_hand_1cls_test.strip()

with open('hand_1cls_test.txt', "w") as f :
  f.write(str_hand_1cls_test)

In [0]:
with open('hand.names', "w") as f :
  f.write("hand")

In [0]:
with open('hand_1cls.data', "w") as f :
  f.write("""classes=1
train=./data/hand_1cls_train.txt
valid=./data/hand_1cls_test.txt
names=data/hand.names
backup=backup/
eval=coco""")

# Clone Yolo

In [0]:
!git clone https://github.com/ultralytics/yolov3  # clone
%cd yolov3

## Update Config (optional)

for 1 classes, `.cfg` file already made by ultralytics

Just Use `cfg.yolo3-spp-1cls.cfg`

For other number of class: 

`n` is your class count
**Update *.cfg file**  

* `filters=255 -> filters=[4 + 1 + n] * 3`
* `classes=80 -> classes=n`

![Yolo Config](https://user-images.githubusercontent.com/26833433/66830924-e03f9500-ef56-11e9-9d09-97f9921cab39.png)

# Move data files 

![teks alternatif](https://user-images.githubusercontent.com/26833433/55683084-f9c6ab00-593b-11e9-877d-9003afa44aa1.png)

In [0]:
!pwd

In [0]:
!cp ../hand.names ./data/
!cp ../hand_1cls.data ./data/
!cp ../hand_1cls_test.txt ./data/
!cp ../hand_1cls_train.txt ./data/

# Train

In [0]:
!python train.py --epochs 10 --data data/hand_1cls.data --cfg cfg/yolov3-spp-1cls.cfg --single-cls

In [0]:
!python train.py --epochs 50 --data data/hand_1cls.data --cfg cfg/yolov3-spp-1cls.cfg --single-cls

# Save Model

In [0]:
# Download but most likely fail because large size

# from google.colab import files
# files.download('./weights/best.pt') 


In [0]:
# Connect to gdrive

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# copy best model
# !cp /content/yolov3/weights/best.pt "/content/drive/My Drive/Models weights/hand detection yolov3 own train/"

In [0]:
# rename model name  
# mv "/content/drive/My Drive/Models weights/hand detection yolov3 own train/best.pt" "/content/drive/My Drive/Models weights/hand detection yolov3 own train/yolov3_epochs_100.pt"

# Detect New Picture




## Copy Photos

Copy test photos to `/content/yolov3/data/samples`

In [0]:
!cp /content/hand/images/test/*.jpg /content/yolov3/data/samples/

## Run detect.py

In [0]:
!python detect.py --weights weights/best.pt --cfg cfg/yolov3-spp-1cls.cfg --names data/hand.names

## Show Result

In [0]:
Image.open(os.path.join('output', 'zidane.jpg')).resize((400,300))

In [0]:
np.random.seed(100)
np.random.choice(os.listdir("output"), 10, replace = False)

In [0]:
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 10))

np.random.seed(234)
list_filename = np.random.choice(os.listdir("output"), 10, replace = False)

for img_loc, filename in enumerate(list_filename):
    img = Image.open(os.path.join('output', filename))
    
    ax = fig.add_subplot(2, 5, img_loc+1, xticks=[], yticks=[])
    plt.imshow(img , cmap = 'gray')
    plt.title(f"{filename}")

# Convert Model from pytorch to Darknet

Advantages: model size reduced ~2x 

In [0]:
!python -c "from models import *; convert('cfg/yolov3-spp-1cls.cfg', 'weights/best.pt')"

In [0]:
# copy best model
!cp /content/yolov3/converted.weights "/content/drive/My Drive/Models weights/hand detection yolov3 own train/"

In [0]:
# rename model name  
mv "/content/drive/My Drive/Models weights/hand detection yolov3 own train/converted.weights" "/content/drive/My Drive/Models weights/hand detection yolov3 own train/yolov3_epochs_100.weights"

# Redetect using Darknet Model

In [0]:
!rm -r output

In [0]:
!python detect.py --weights converted.weights --cfg cfg/yolov3-spp-1cls.cfg --names data/hand.names

In [0]:
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 10))

np.random.seed(234)
list_filename = np.random.choice(os.listdir("output"), 10, replace = False)

for img_loc, filename in enumerate(list_filename):
    img = Image.open(os.path.join('output', filename))
    
    ax = fig.add_subplot(2, 5, img_loc+1, xticks=[], yticks=[])
    plt.imshow(img , cmap = 'gray')
    plt.title(f"{filename}")

The result is same, so far for this 10 sample :D

# Thank You